<a href="https://colab.research.google.com/github/simon-m-mudd/physical_geography_practicals/blob/main/physical_geography_flood_practical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Physical Geography practical 1: Analysis of flood data (and your first look at python)

Written by Simon M Mudd, last update 31-Jan-2024

## Overview

In this practical we will work with some data that comes from the National River Flow Archive (NFRA) which can be found at: https://nrfa.ceh.ac.uk/

The practical shows you how to perform flood frequency analysis, and then look at the results.

It will also give you a basic overview of python. We will use this again later in the course.

## What is this notebook?

You are reading this practical in something called an ipython notebook. The "i" is for "interactive", the "python" is a programming language.

If you are doing a degree in the School of GeoSciences at the University of Edinburgh, you will take some classes about the python programming language. This is not one of those courses.

In this course I just want you to look at data. In the past we have used spreadsheet programs (i.e., excel) for this. But graphs made here look better, and these notebooks are more flexible.

### The only thing you need to know about python programming at this stage

This notebook is organised into "cells".

* Some of the cells just contain text.
* Some contain python code that does something

You don't change the text cells.
For the python cells you just make them run.

**In Noteable** python cells have **In []:** written next to them.

**In Google colab** python cells have a symbol that looks like a "play" button if you hover over them.

In both of these environments, if you click on the cell and then type `shift+enter` the cell will run.


In [ ]:
print("You just ran some python code! You can now put `Data Scientist` into you Linkedin profile!")

## On to the practial!

### What is the NRFA

You can read the website: https://nrfa.ceh.ac.uk/

But the NRFA contains data from gauging stations distributed around the UK. A gauging station is a place where hydrologists measure flow. Usually they do some very labour intensive measurements of discharge (that is, the volume of water per unit of time, usually reported in m$^3$/s), and then relate this to the "stage" of the river (that is, the elevation of the water surface).

The relationship between the stage and the discharge is called a **rating curve**. Hydrolgists make rating curves because stage is much easier to measure than discharge.

You can find out the data about the gauging stations in the NRFA catalogue. But for now all you need to know is each station has a **station ID**, which is just a number.

For example, station 8010 is the River Spey at Grantown

### Get some data from the NFRA!

Now lets get some data from the National River Flow archive (NRFA).

We use a little python tool for this (you need to be connected to the internet).

The next bit of code installs the tool. Run the next cell.

In [ ]:
!pip install nrfapy &> /dev/null

Now lets get some data.

We are going to get the annual maximum flow. This is the peak discharge at the station for that water year. The standard UK water-year begins on 1st October: for example the 1999 water year begins on 1/10/1999 and ends on 30/9/2000. Choosing the end of September as the end of the water year avoids splitting the data series at a flood-prone time of year.

In the following example I am getting the station number 8010 (see above) and I am getting the `amax-flow` data, which is the annual maximum flow. There are other options but you don't need to worry about them.

In [ ]:
import nrfapy

station = 8010
data_type = "amax-flow"
amax_flow_df = nrfapy.get_ts(station,data_type)
amax_flow_df.head()

Just to develop some physical intuition about discharge: a lowland river might flow 1 metre per second at low flows, and you might get over 3 m/s during a flood. The station on this part to the Spey is 60m wide. For a discharge of 300 m$^3$/s, a velocity of 3m/s, and a width of 60m, you would get a water depth of 1.66m. We can look at the stage measurement (how deep the river is during the flood) to check our estimates:

In [ ]:
data_type = "amax-stage"
amax_stage_df = nrfapy.get_ts(station,data_type)
amax_stage_df.head()

Hey, not bad!

We can also plot the maximum disharge from each year's flood.

Again, you don't need to change anything below, just click on the cells and type shift+enter to run them.

In [ ]:
import pandas as pd

# Convert 'datetime' column to datetime format
amax_flow_df['datetime'] = pd.to_datetime(amax_flow_df['time'], format='%Y-%m-%dT%H:%M:%S')


# Extract the year from the datetime column
amax_flow_df['year'] = amax_flow_df['datetime'].dt.year

amax_flow_df.head()

In [ ]:
import matplotlib.pyplot as plt

# This makes a bar plot
ax = amax_flow_df.plot.bar(x='year', y='amax-flow', rot=90, figsize=(9, 5))

# Set x-axis label to be the year
plt.xlabel('Year')

# Set y-axis label
plt.ylabel('Maximum annual flood (m$^3$/s)')

# Set the title
plt.title('Maximum annual flood for station number: '+str(station))

# Show every 3rd label on the x-axis
plt.xticks(amax_flow_df["amax-flow"].index[::3]);

### Now for flood frequency

Now we are going to look at the flood frequency.

Before we do the formal flood frequency analysis we will calculate some simple statistics of the floods. First lets look at the probability of annual maximum discharges.

We do this using something called a kernal density estimation, which is a method to estimate the probability density of a dataset.

In [ ]:
ax = amax_flow_df['amax-flow'].plot.kde(bw_method=1)

# Set x-axis label to be the year
plt.xlabel('Discharge (m$^3$/s)')

# Set the title
plt.title('Probability density of annual maximum discharge (in m$^3$/s) for station number: '+str(station))

We can also get the average and median values

In [ ]:
mean_amax_Q = amax_flow_df['amax-flow'].mean()
median_amax_Q = amax_flow_df['amax-flow'].median()
print("The mean annual maximum flow is: " +str(mean_amax_Q) + " m^3/s and the median is: " + str(median_amax_Q)+ " m^3/s")

To do flood frequency analysis, we need to rank the floods, with the largest flood ranked 1, the second largest 2, and so on. There is a `pandas` script for that:

In [ ]:
amax_flow_df['rank_column'] = amax_flow_df['amax-flow'].rank(ascending=False).astype(int)
amax_flow_df.head(10)

We need to know how many records there are, the ode for that is below.

In [ ]:
n_records = len(amax_flow_df['amax-flow'])
print("The number of years on record is: "+str(n_records))


#### Calculating return period

The return period, *R*, is calculated using the following equation:

$R = \frac{n+1}{n}$

where *R* is the return period (in years), *n* is the number of years on record, and *m* is the rank of the flood in the *n* year record.

So we need to divide (*n*+1) (for station 8010 this is 71) by each year's rank, which again is quite easy using `pandas`:

In [ ]:
amax_flow_df['return_period'] = (n_records+1)/amax_flow_df['rank_column']
amax_flow_df.head(10)

#### Make a flood frequency plot

Lets now plot the return period against the discharge:

In [ ]:
import matplotlib.pyplot as plt

# This makes a bar plot
ax = amax_flow_df.plot.scatter(x='return_period', y='amax-flow', figsize=(9, 5))

# Set x-axis label to be the year
plt.xlabel('Maximum annual flood (m$^3$/s)')

# Set y-axis label
plt.ylabel('Return period (yrs)')

# Set the title
plt.title('Return periods for station number: '+str(station))


This actually looks better if you set the x axis to be in logarithmic scale.

In [ ]:
import matplotlib.pyplot as plt

# This makes a bar plot
ax = amax_flow_df.plot.scatter(x='return_period', y='amax-flow', figsize=(9, 5))

# Set x-axis label to be the year
plt.xlabel('Maximum annual flood (m$^3$/s)')

# Set y-axis label
plt.ylabel('Return period (yrs)')

# Set the title
plt.title('Return periods for station number: '+str(station))

# Set x-axis to logarithmic scale
ax.set_xscale('log')

We can fit that data with an equation of the form:

$Q_{flood} = a \ln (R)+b$

where ln is the natural logarithm and a and b are some constants you fit.

We can do that in `pandas` and by using some python statistics tools.

In [ ]:
import numpy as np
from scipy.stats import linregress
import matplotlib.pyplot as plt

# This makes a scatter plot
ax = amax_flow_df.plot.scatter(x='return_period', y='amax-flow', figsize=(9, 5))

# Set x-axis label
plt.xlabel('Maximum annual flood (m$^3$/s)')

# Set y-axis label
plt.ylabel('Return period (yrs)')

# Set the title
plt.title('Return periods for station number: '+str(station))

# Set x-axis to logarithmic scale
ax.set_xscale('log')

# Perform a linear regression on the natural log of the x data
slope, intercept, r_value, p_value, std_err = linregress(np.log(amax_flow_df['return_period']), amax_flow_df['amax-flow'])

# Create a range of x values for the fitted line
x_fit = np.linspace(min(amax_flow_df['return_period']), max(amax_flow_df['return_period']), num=500)

# Calculate the corresponding y values for the fitted line
y_fit = slope * np.log(x_fit) + intercept

# Plot the fitted line
plt.plot(x_fit, y_fit, 'r', label=f'fit: y = {slope:.2f} * ln(x) + {intercept:.2f}')

# Show the legend
plt.legend()

# Show the plot
plt.show()


At this point you may be looking at this code and wondering "how on Earth will I be able to remember all these python things?"

The author of this notebook is not bad at python but as recently as 2022 would need to spend some time refreshing his memory on Stackoverflow. But now we have AI: I just had the code from the previous cell and asked Microsoft Bard this question:

*How do I add a trendline to that plot. I want to fit it with a natural log regression?*

Unless you are doing something quite niche, AI can write functional python code for basic data analysis.

#### Find the return period of the mean annual flood

Okay, lets get the return period of the mean annual flood. The mean annual flood is specifically the mean value of all the amax-flow data, which you calculated earlier. We can calulate it by reorganising the regression equation:


$\frac{Q_{flood}-b}{a} = \ln (R)$

which with some more algebra is
$R = exp[ \frac{Q_{flood}-b}{a}]$

In [ ]:
R_maf = np.exp( (mean_amax_Q - intercept) / slope)
print("The return period of the mean annual flood is: "+str(R_maf))

And we can also predict much bigger floods, by taking that original equation:

$Q_{flood} = a \ln (R)+b$

and plugging in big return periods. Lets see how big a 500 year flood is:

In [ ]:
big_return_period = 500
flood_big = slope*np.log(big_return_period)+ intercept
print("The discharge of a flood with a return period of "+str(big_return_period)+ " years is "+str(flood_big) + "m^3/s")

## Getting information about the stations

So we have looked at a single station (station number 8010, the River Spey at Grantown). You might want to search for stations on your favourite river. For this we need to look at the NRFA catalogue.

We can get this with a line of python: `stations_data = nrfapy.catalogue()`

After that we use `pandas` to search the data. The `head()` command below shows you the first few lines of the data and what the column names are.

In [ ]:
import nrfapy
stations_data = nrfapy.catalogue()
stations_data.head()

You can search for an individual station by using the line below.

Try to change the number from 8010.

If you use a number that does not correspond to a station you will get an empty row.

**Do not change anything else in the code cell!**

In [ ]:
stations_data.loc[stations_data['id'] == 8010]

You can then use the below query to get all the stations on your favourite river.

Try changing the name of the river!

In [ ]:
stations_data[stations_data['river'].str.contains("Spey")]

### Plot station locations

We might want to see where these stations are.

We can plot them on a map, but first we need to install something called contextily, which is a nice package for showing basemaps. You don't need to worry about that at this point.

In [ ]:
!pip install contextily &> /dev/null

In [ ]:
import geopandas as gpd
import contextily as ctx

# Make a dataframe with just the stations on the Spey.
my_stations = stations_data[stations_data['river'].str.contains("Spey")]

# Now we use the British National Grid coordinates
# Every coordinate reference system (crs) has a code. The british national grid's code is EPSG:27700
gdf2 = gpd.GeoDataFrame(my_stations, geometry=gpd.points_from_xy(my_stations.easting, my_stations.northing), crs='EPSG:27700')

# You could also do this with latitude and longitude, but it results in a slightly distorted map
#gdf = gpd.GeoDataFrame(my_stations, geometry=gpd.points_from_xy(my_stations.longitude, my_stations.latitude), crs='EPSG:4326')


In [ ]:
# Create a plot using geopands
ax = gdf2.plot(marker='o', color='red', markersize=20, figsize=(9, 9))


# This labels the stations
for x, y, label in zip(gdf2.geometry.x, gdf2.geometry.y, gdf2.name):
    ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points")

# The plot looks better if you pad the points by 30km
# The next three lines to that. If you want to change the padding only change
# the "padding" number
padding = 30000
xlim = (gdf2.total_bounds[0] - padding, gdf2.total_bounds[2] + padding)
ylim = (gdf2.total_bounds[1] - padding, gdf2.total_bounds[3] + padding)
ax.set_xlim(xlim)
ax.set_ylim(ylim)

# Now add a basemap
ctx.add_basemap(ax,source=ctx.providers.OpenStreetMap.Mapnik,crs=gdf2.crs.to_string())

# Set axis labels and title
ax.set_xlabel('Easting (m)')
ax.set_ylabel('Northing (m)')
ax.set_title('NRFA selected station locations')

# Show the plot
plt.show()

If you want to play with this try doing all the above but just change the river name.

## Look at a few of your favourite rivers

Now you will test your ability to look at your own favourite river.

The steps are:
1. Search the catalog for the correct station number.
2. Change the station number when getting the data.
2. Make the rank column.
3. Get the mean annual flood.
3. Get the number of years on record from the data.
4. Calculate the return period.
5. Plot the return period and get the function that describes the flow as a function of return period.
6. Calculate the return period of the mean annual flood.
7. Get the flow of a big flood (say 500 year).


In [ ]:
my_river = "Feshie"
stations_data[stations_data['river'].str.contains(my_river)]

Change the station number below.

In [ ]:
import nrfapy

station = 8013
data_type = "amax-flow"
amax_flow_df = nrfapy.get_ts(station,data_type)
amax_flow_df.head()

Make the rank column

In [ ]:
amax_flow_df['rank_column'] = amax_flow_df['amax-flow'].rank(ascending=False).astype(int)
amax_flow_df.head(10)

Get the number of years on record

In [ ]:
n_records = len(amax_flow_df['amax-flow'])
print("The number of years on record is: "+str(n_records))


Get the mean annual flood

In [ ]:
mean_amax_Q = amax_flow_df['amax-flow'].mean()
median_amax_Q = amax_flow_df['amax-flow'].median()
print("The mean annual maximum flow is: " +str(mean_amax_Q) + " m^3/s and the median is: " + str(median_amax_Q)+ " m^3/s")

Calculate the return period

In [ ]:
amax_flow_df['return_period'] = (n_records+1)/amax_flow_df['rank_column']
amax_flow_df.head(10)

Calculate the fit on the return period

In [ ]:
import numpy as np
from scipy.stats import linregress
import matplotlib.pyplot as plt

# This makes a scatter plot
ax = amax_flow_df.plot.scatter(x='return_period', y='amax-flow', figsize=(9, 5))

# Set x-axis label
plt.xlabel('Maximum annual flood (m$^3$/s)')

# Set y-axis label
plt.ylabel('Return period (yrs)')

# Set the title
plt.title('Return periods for station number: '+str(station))

# Set x-axis to logarithmic scale
ax.set_xscale('log')

# Perform a linear regression on the natural log of the x data
slope, intercept, r_value, p_value, std_err = linregress(np.log(amax_flow_df['return_period']), amax_flow_df['amax-flow'])

# Create a range of x values for the fitted line
x_fit = np.linspace(min(amax_flow_df['return_period']), max(amax_flow_df['return_period']), num=500)

# Calculate the corresponding y values for the fitted line
y_fit = slope * np.log(x_fit) + intercept

# Plot the fitted line
plt.plot(x_fit, y_fit, 'r', label=f'fit: y = {slope:.2f} * ln(x) + {intercept:.2f}')

# Show the legend
plt.legend()

# Show the plot
plt.show()


In [ ]:
R_maf = np.exp( (mean_amax_Q - intercept) / slope)
print("The return period of the mean annual flood is: "+str(R_maf))

I've set this to the 100 year flood. Don't change it because you will look at the 100 year flood in the next step.

In [ ]:
big_return_period = 100
flood_big = slope*np.log(big_return_period)+ intercept
print("The discharge of a flood with a return period of "+str(big_return_period)+ " years is "+str(flood_big) + "m^3/s")

## Last steps

One of problem with this form of flood frequency analysis is that it is very sensitive to the number of years on record.  

In the next cell, set the station to the same station as above, and select the number of years to remove from the record.

Then just run all the cells below and see if the 100 year flood discharge is different.

In [ ]:
years_to_remove = 20
station = 8013
data_type = "amax-flow"
amax_flow_df_new = nrfapy.get_ts(station,data_type)
amax_flow_df_short = amax_flow_df_new.iloc[:-years_to_remove]
amax_flow_df_short['rank_column'] = amax_flow_df_short['amax-flow'].rank(ascending=False).astype(int)
n_records = len(amax_flow_df_short['amax-flow'])
print("The number of years on record is: "+str(n_records))

In [ ]:
amax_flow_df_short['return_period'] = (n_records+1)/amax_flow_df_short['rank_column']
amax_flow_df_short.head(10)

In [ ]:
import numpy as np
from scipy.stats import linregress
import matplotlib.pyplot as plt

# This makes a scatter plot
ax = amax_flow_df_short.plot.scatter(x='return_period', y='amax-flow', figsize=(9, 5))

# Set x-axis label
plt.xlabel('Maximum annual flood (m$^3$/s)')

# Set y-axis label
plt.ylabel('Return period (yrs)')

# Set the title
plt.title('Return periods for station number: '+str(station))

# Set x-axis to logarithmic scale
ax.set_xscale('log')

# Perform a linear regression on the natural log of the x data
sslope, sintercept, sr_value, sp_value, sstd_err = linregress(np.log(amax_flow_df_short['return_period']), amax_flow_df_short['amax-flow'])

# Create a range of x values for the fitted line
x_fit = np.linspace(min(amax_flow_df_short['return_period']), max(amax_flow_df_short['return_period']), num=500)

# Calculate the corresponding y values for the fitted line
y_fit = sslope * np.log(x_fit) + sintercept

# Plot the fitted line
plt.plot(x_fit, y_fit, 'r', label=f'fit: y = {sslope:.2f} * ln(x) + {sintercept:.2f}')

# Show the legend
plt.legend()

# Show the plot
plt.show()


In [ ]:
big_return_period = 100
flood_big = sslope*np.log(big_return_period)+ sintercept
print("The discharge of a flood with a return period of "+str(big_return_period)+ " years is "+str(flood_big) + "m^3/s")

I suggest you find a few rivers to test sensitivity to the flow record. What kind of rivers are most sensitive?